In [1]:
import ruleminer
import logging
import sys
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
logging.basicConfig(
    stream=sys.stdout, format="%(asctime)s %(message)s", level=logging.INFO
)

In [3]:
df = pd.DataFrame(
    columns=[
        "Name",
        "Type",
        "Assets",
        "TV-life",
        "TV-nonlife",
        "Own funds",
        "Excess",
    ],
    data=[
        ["Insurer1", "life insurer", 1000, 800, 0, 200, 200],
        ["Insurer2", "non-life insurer", 4000, 0, 3200, 800, 800],
        ["Insurer3", "non-life insurer", 800, 0, 700, 100, 100],
        ["Insurer4", "life insurer", 2500, 1800, 0, 700, 700],
        ["Insurer5", "non-life insurer", 2100, 0, 2200, 200, 200],
        ["Insurer6", "life insurer", 9000, 8800, 0, 200, 200],
        ["Insurer7", "life insurer", 9000, 8800, 0, 200, 200],
        ["Insurer8", "life insurer", 9000, 8800, 0, 200, 200],
        ["Insurer9", "non-life insurer", 9000, 8800, 0, 200, 200],
        ["Insurer10", "non-life insurer", 9000, 0, 8800, 200, 199.99],
    ],
)

In [4]:
df

,Name,Type,Assets,TV-life,TV-nonlife,Own funds,Excess
0,Insurer1,life insurer,1000,800,0,200,200.00
1,Insurer2,non-life insurer,4000,0,3200,800,800.00
2,Insurer3,non-life insurer,800,0,700,100,100.00
3,Insurer4,life insurer,2500,1800,0,700,700.00
4,Insurer5,non-life insurer,2100,0,2200,200,200.00
5,Insurer6,life insurer,9000,8800,0,200,200.00
6,Insurer7,life insurer,9000,8800,0,200,200.00
7,Insurer8,life insurer,9000,8800,0,200,200.00
8,Insurer9,non-life insurer,9000,8800,0,200,200.00
9,Insurer10,non-life insurer,9000,0,8800,200,199.99


In [5]:
# templates = [{'expression': 'if (((?P<col>[{".*"}]) == ".*") or ((?P=col)==".*")) then ({".*"} == 0)'}]
# r = ruleminer.RuleMiner(templates=templates, data=df)

In [6]:
# import re
# a = re.search('.*?P<([a-z]+)>.*', 'if (((?P<col>{".*"}) == ".*") or ((?P=col)==".*")) then ({".*"} == 0)')
# print(a.group(1))

In [7]:
templates = [{"expression": 'if ({".*"} == ".*") then ({".*"} == 0)'}]
r = ruleminer.RuleMiner(templates=templates, data=df)
r.rules.to_dict()
# [[('{".*"}', None)]]

2023-08-29 08:39:30,635 Expression for if-part ([['{".*"}', '==', '".*"']]) generated
2023-08-29 08:39:30,669 Finished


{'rule_id': {0: 0, 1: 1},
 'rule_group': {0: 0, 1: 0},
 'rule_definition': {0: 'if({"Type"}=="life insurer")then({"TV-nonlife"}==0)',
  1: 'if({"Type"}=="non-life insurer")then({"TV-life"}==0)'},
 'rule_status': {0: '', 1: ''},
 'abs support': {0: 5, 1: 4},
 'abs exceptions': {0: 0, 1: 1},
 'confidence': {0: 1.0, 1: 0.8},
 'encodings': {0: {}, 1: {}}}

In [8]:
r.rules = pd.DataFrame(
    {
        "rule_id": {0: 0, 1: 1},
        "rule_group": {0: 0, 1: 0},
        "rule_definition": {
            0: 'if({"Type"}=="life insurer")then({"TV-nonlife"}==0)',
            1: 'if({"Type"}==abs("non-life insurer"))then({"TV-life"}==0)',
        },
        "rule_status": {0: "", 1: ""},
        "abs support": {0: 5, 1: 4},
        "abs exceptions": {0: 0, 1: 1},
        "confidence": {0: 1.0, 1: 0.8},
        "encodings": {0: {}, 1: {}},
    }
)

In [9]:
r.evaluate()

2023-08-29 08:39:31,594 Error when evaluating rule results for rule id 1


,rule_id,rule_group,rule_definition,rule_status,abs support,abs exceptions,confidence,result,indices
0,0,0,"if({""Type""}==""life insurer"")then({""TV-nonlife""...",,5,0,1.0,True,0
1,0,0,"if({""Type""}==""life insurer"")then({""TV-nonlife""...",,5,0,1.0,True,3
2,0,0,"if({""Type""}==""life insurer"")then({""TV-nonlife""...",,5,0,1.0,True,5
3,0,0,"if({""Type""}==""life insurer"")then({""TV-nonlife""...",,5,0,1.0,True,6
4,0,0,"if({""Type""}==""life insurer"")then({""TV-nonlife""...",,5,0,1.0,True,7


In [10]:
# templates = [{'expression': 'if ({".*"} in [".*"]) then ({".*"} == 0)'}]
# r = ruleminer.RuleMiner(templates=templates, data=df)
# r.rules

In [ ]:
df

In [ ]:
templates = [{"expression": '({"Own funds"} <= quantile({"Own funds"}, 0.95))'}]
r = ruleminer.RuleMiner(templates=templates, data=df)
r.rules

In [ ]:
templates = [{"expression": '({"Own funds"} <= quantile({"Own funds"}, 0.95))'}]
r = ruleminer.RuleMiner(
    templates=templates, data=df, params={"evaluate_quantile": True}
)
r.rules

In [ ]:
templates = [{"expression": '(max({".*"}, {".*"})=={".*"})'}]
r = ruleminer.RuleMiner(templates=templates, data=df)

In [ ]:
r.rules

In [ ]:
params = {
    "metrics": [
        "rule power factor",
        "added value",
        "abs support",
        "abs exceptions",
        "confidence",
    ]
}
templates = [{"expression": 'if ({".*"} == ".*") then ({"TV.*"} > 0)'}]
r = ruleminer.RuleMiner(templates=templates, data=df, params=params)

In [ ]:
r.rules

In [ ]:
r.evaluate()

In [ ]:
templates = [{"group": "simple", "expression": '({".*"} >= 0)'}]
r = ruleminer.RuleMiner(templates=templates, data=df)
r.rules

In [ ]:
templates = [{"group": "simple", "expression": '({".*"} == {".*"})'}]
r = ruleminer.RuleMiner(templates=templates, data=df, params={"decimal": 3})
r.rules

In [ ]:
templates = [{"group": "simple", "expression": '(({".*"} + {".*"} * {".*"})=={".*"})'}]
r = ruleminer.RuleMiner(templates=templates, data=df)
r.rules

In [ ]:
templates = [
    {"group": "simple", "expression": 'if ({"Type"} == ".*") then ({".*"} == 0)'},
    {"group": "complex", "expression": 'if ({".*"} == ".*") then ({".*"} == 0)'},
]
r = ruleminer.RuleMiner(templates=templates, data=df)
r.rules

In [ ]:
ruleminer.flatten_and_sort(
    ruleminer.parser.RULE_SYNTAX.parse_string(
        '(({"4"} + {"3"} + {"2"} + {"1"}) > 0)', parse_all=True
    ).as_list()
)

In [ ]:
templates = [
    {
        "expression": 'if (({"T.*"} == ".*") & ({"T.*"} == 0) & ({"T.*"} == 0) & ({"T.*"} == 0)) then ({"TV.*"} > 0)'
    }
]
r = ruleminer.RuleMiner(
    templates=templates,
    data=df,
    params={
        "metrics": ["casual confidence", "abs support", "abs exceptions", "confidence"]
    },
)

In [ ]:
r.rules

In [ ]:
templates = [{"expression": '({".*"} > {".*"})'}]
r = ruleminer.RuleMiner(
    templates=templates,
    data=df,
    params={"metrics": ["support", "abs support", "confidence"]},
)
r.generate()

In [ ]:
r.rules

In [ ]:
templates = [{"expression": '(({".*"}+{".*"}) == {".*"})'}]
r = ruleminer.RuleMiner(templates=templates, data=df)

In [ ]:
r.rules

In [ ]:
templates = [
    {"group": "simple", "expression": 'if ({"Type"} == ".*") then ({".*"} == 0)'},
    {"group": "complex", "expression": 'if ({".*"} == ".*") then ({".*"} == 0)'},
]
r = ruleminer.RuleMiner(
    templates=templates, data=df, params={"result_expressions": True}
)
r.evaluate()
assert r is not None

In [ ]:
r.rules

In [ ]:
templates = [
    {
        "group": "simple",
        "expression": 'if ({"Name"} == "Insurer2") then ({"Type"} == ".*")',
    }
]
r = ruleminer.RuleMiner(
    templates=templates, data=df, params={"result_expressions": True}
)
r.evaluate()
assert r is not None

In [ ]:
r.rules